# Comparing Cython, Numpy and Numba for a PFR Distributed Model

## The model

Consider a simple PFR reactor as follows:

![](pfr_scheme.png)

The concentration inside the reactor can be described in the position $z$ and time $t$ by the partial differential equation:

$$\frac{\partial C(t,z)}{\partial t} + v_z \frac{\partial C(t,z)}{\partial z} - D \frac{\partial^2 C(t,z)}{\partial t^2} - r(t,z) = 0$$

$$C(0,z) = C_{ini}(z)$$

$$C(t,0) - \frac{D}{v_z}\frac{d C(t,z)}{dz}\Big\vert_{z=0} = C_f(t)$$

$$\frac{D}{v_z}\frac{d C(t,z)}{dz}\Big\vert_{z=L} = 0$$

An approach to solve this PDE is discretizing the $z$ domain and approximating the derivatives using finite differences. Doing so we obtain the final set of equation as an ordinary differential equation set in the form $\frac{d \mathbf{C}}{d t} = f(\mathbf{C})$.

$$\frac{dC_1(t)}{dt} = \frac{D}{h^2}(C_2 - 2C_1 + C_0) - \frac{v_z}{2h}(C_2-C_0) + r_1$$

$$\frac{dC_i(t)}{dt} = \frac{D}{h^2}(C_{i+1} - 2C_{i} + C_{i-1}) - \frac{v_z}{2h}(C_{i+1}-C_{i-1}) + r_{i} \text{ for } i=2,\ldots, N-1 $$ 

$$\frac{dC_N(t)}{dt} = \frac{D}{h^2}(C_{N+1} - 2C_{N} + C_{N-1}) - \frac{v_z}{2h}(C_{N+1}-C_{N-1}) + r_{N}$$

with:

$$C_0 = \left( 1 + \frac{D}{v_z h}\right)^{-1}\left(\frac{D}{v_z h} C_1 + C_f \right)$$

$$C_{N+1}=C_N$$

In the next, this problem is solved using python, cython, numpy and numba. Moreover, a performance comparison is provided for various values of the number of discretization points.

In [1]:
import numpy as np
import dasslcy
from functools import partial
import perfplot
import numba
NPTS = 100
dyn_solver = partial(dasslcy.solve, share_res=1)
%load_ext Cython

## Class for the model data

In [2]:
class pfr():
    def __init__(self, N = 20):
        self.D = 1.0
        self.vz = 1.0
        self.k = 1.0
        self.Cf = 1.0
        self.z0 = 0.0
        self.zf = 1.0
        self.N = N
        self.h = self.get_h()

    def get_h(self):
        return (self.zf - self.z0) / self.N
    
def solver_setup_base(N):
    par = pfr(N)
    t0 = np.array([5.0])
    y0 = np.zeros(par.N)
    yp0 = None
    atol = 1e-8
    rtol = 1e-6
    return [t0, y0, yp0, par, rtol, atol]

## Pure Python

Setting dynamic model and solving

In [3]:
def model_pfr(t, y, yp, par, res):
    N = par.N
    D, vz, k, Cf, h = par.D, par.vz, par.k, par.Cf, par.h
    dCi = yp
    Ci = y
    aux1 = D / (vz * h)
    C0 = 1.0 / (1.0 + aux1) * (aux1 * Ci[0] + Cf)
    CNp1 = Ci[N - 1]
    aux2 = D / h**2
    aux3 = vz / (2 * h)
    res[0] = aux2 * (Ci[1] - 2.0 * Ci[0] + C0) - \
        aux3 * (Ci[1] - C0) + k * Ci[0] - dCi[0]
    for i in np.arange(1, N - 1):
        tt1 = aux2 * (Ci[i + 1] - 2.0 * Ci[i] + Ci[i - 1])
        tt2 = -aux3 * (Ci[i + 1] - Ci[i - 1]) + k * Ci[i]
        res[i] = tt1 + tt2 - dCi[i]
    res[N - 1] = aux2 * (CNp1 - 2.0 * Ci[N - 1] + Ci[N - 2]) - \
        aux3 * (CNp1 - Ci[N - 2]) + k * Ci[N - 1] - dCi[N - 1]
    return res, 0

In [4]:
base_args = solver_setup_base(NPTS)
o = %timeit -r 10 -n 1 -o dyn_solver(model_pfr, *base_args) 

474 ms ± 156 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


## Numpy broadcasting

In [5]:
def model_pfr_np(t, y, yp, par, res):
    N = par.N
    D, vz, k, Cf, h = par.D, par.vz, par.k, par.Cf, par.h
    dCi = yp
    Ci = y
    aux1 = D / (vz * h)
    C0 = 1.0 / (1.0 + aux1) * (aux1 * Ci[0] + Cf)
    CNp1 = Ci[N - 1]
    aux2 = D / h**2
    aux3 = vz / (2 * h)
    res[0] = aux2 * (Ci[1] - 2.0 * Ci[0] + C0) - \
        aux3 * (Ci[1] - C0) + k * Ci[0] - dCi[0]
    tt1 = aux2 * (Ci[2:] - 2.0 * Ci[1:-1] + Ci[0:-2])
    tt2 = -aux3 * (Ci[2:] - Ci[0:-2]) + k * Ci[1:-1]
    res[1:-1] = tt1 + tt2 - dCi[1:-1]

    res[N - 1] = aux2 * (CNp1 - 2.0 * Ci[N - 1] + Ci[N - 2]) - \
        aux3 * (CNp1 - Ci[N - 2]) + k * Ci[N - 1] - dCi[N - 1]
    return res, 0

In [6]:
o = %timeit -r 10 -n 1 -o dyn_solver(model_pfr_np, *base_args)

45.3 ms ± 7.35 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


## Cython Naive implementation

In [7]:
%%cython
import numpy as np
cimport numpy as np
def model_pfr_cy(t, y, yp, par, res):
    N = par.N
    D, vz, k, Cf, h = par.D, par.vz, par.k, par.Cf, par.h
    dCi = yp
    Ci = y
    aux1 = D / (vz * h)
    C0 = 1.0 / (1.0 + aux1) * (aux1 * Ci[0] + Cf)
    CNp1 = Ci[N - 1]
    aux2 = D / h**2
    aux3 = vz / (2 * h)
    res[0] = aux2 * (Ci[1] - 2.0 * Ci[0] + C0) - \
        aux3 * (Ci[1] - C0) + k * Ci[0] - dCi[0]
    for i in np.arange(1, N - 1):
        tt1 = aux2 * (Ci[i + 1] - 2.0 * Ci[i] + Ci[i - 1])
        tt2 = -aux3 * (Ci[i + 1] - Ci[i - 1]) + k * Ci[i]
        res[i] = tt1 + tt2 - dCi[i]
    res[N - 1] = aux2 * (CNp1 - 2.0 * Ci[N - 1] + Ci[N - 2]) - \
        aux3 * (CNp1 - Ci[N - 2]) + k * Ci[N - 1] - dCi[N - 1]
    return res, 0

In [8]:
o = %timeit -r 10 -n 1 -o dyn_solver(model_pfr_cy, *base_args)

393 ms ± 24.5 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


## Cython Typed Implementation

In [9]:
%%cython
import numpy as np
cimport numpy as np
cimport cython 

#cython: boundscheck=False
#cython: wraparound=False

cdef class Pfr_Cython:
    cdef:
        double D, vz, k, Cf, z0, zf, h
        int N
    def __init__(self, N = 20):
        self.N = N
        self.D = 1.0
        self.vz = 1.0
        self.k = 1.0
        self.Cf = 1.0
        self.z0 = 0.0
        self.zf = 1.0
        self.h = self.get_h()
    cdef get_h(self):
        return (self.zf - self.z0) / self.N
    
    @cython.boundscheck(False)  # Deactivate bounds checking
    @cython.wraparound(False)   # Deactivate negative indexing.
    @cython.nonecheck(False)
    @cython.cdivision(True)    
    cdef cythonized_base_model_calculations(self, np.float64_t[:] res, np.float64_t[:] y, np.float64_t[:] yp,
        int N, double D, double vz, double k, double Cf, double h):
        cdef:
            int i
            double tt1, tt2
        cdef np.float64_t[:] dCi = yp
        cdef np.float64_t[:] Ci = y
        cdef double aux1 = D / (vz * h)
        cdef double C0 = 1.0 / (1.0 + aux1) * (aux1 * Ci[0] + Cf)
        cdef double CNp1 = Ci[N - 1]
        cdef double aux2 = D / h**2
        cdef double aux3 = vz / (2 * h)
        res[0] = aux2 * (Ci[1] - 2.0 * Ci[0] + C0) - \
            aux3 * (Ci[1] - C0) + k * Ci[0] - dCi[0]
        for i in range(1, N - 1):
            tt1 = aux2 * (Ci[i + 1] - 2.0 * Ci[i] + Ci[i - 1])
            tt2 = -aux3 * (Ci[i + 1] - Ci[i - 1]) + k * Ci[i]
            res[i] = tt1 + tt2 - dCi[i]
        res[N - 1] = aux2 * (CNp1 - 2.0 * Ci[N - 1] + Ci[N - 2]) - \
            aux3 * (CNp1 - Ci[N - 2]) + k * Ci[N - 1] - dCi[N - 1]
        pass
    
    @cython.boundscheck(False)  # Deactivate bounds checking
    @cython.wraparound(False)   # Deactivate negative indexing. 
    @cython.nonecheck(False)
    @cython.cdivision(True)
    cpdef cython_model(self, double t, np.float64_t[:] y, np.float64_t[:] yp, np.float64_t[:] res):
        self.cythonized_base_model_calculations(
                res, y, yp, self.N, self.D, self.vz, self.k, self.Cf, self.h)
        return res, 0

In [10]:
#initialize_cy_pfr_model(NPTS)
pr_Cy = Pfr_Cython(NPTS)
cy_args = solver_setup_base(NPTS)
cy_args[3] = None
o = %timeit -r 10 -n 1 -o dyn_solver(pr_Cy.cython_model, *cy_args)

The slowest run took 4.56 times longer than the fastest. This could mean that an intermediate result is being cached.
14 ms ± 9.25 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


## Numba

- Create a function closure to define numba scoped variables

In [11]:
spec = [
    ('N', numba.int32),
    ('D', numba.float64),
    ('vz', numba.float64),
    ('k', numba.float64),
    ('Cf', numba.float64),
    ('z0', numba.float64),
    ('zf', numba.float64),
    ('h', numba.float64),
]
Numba_PFR = numba.jitclass(spec)(pfr)
jitted_pfr_model = numba.jit(model_pfr, nopython=True)
numba_pfr = Numba_PFR(NPTS)
numba_args = solver_setup_base(NPTS)
numba_args[3] = numba_pfr
dyn_solver(jitted_pfr_model, *numba_args);

In [12]:
o = %timeit -r 10 -n 1 -o dyn_solver(jitted_pfr_model, *numba_args)

5.11 ms ± 554 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)


## Performance for discretization point 

In [13]:
def setups_all(N):
    base_args = solver_setup_base(N)
    pr_Cy = Pfr_Cython(N)
    cy_args = solver_setup_base(N)
    cy_args[3] = None
    numba_pfr = Numba_PFR(N)
    numba_args = solver_setup_base(N)
    numba_args[3] = numba_pfr      
    return base_args, (pr_Cy, cy_args), numba_args

def run_py(opts_setup):
#     base_args = solver_setup_base(N)
    dyn_solver(model_pfr, *opts_setup[0])
    return 0

# def setup_np(N):
#     return solver_setup_base(N)

def run_np(opts_setup):
    dyn_solver(model_pfr_np, *opts_setup[0])
    return 0

def run_cy_naive(opts_setup):
    #base_args = solver_setup_base(N)
    dyn_solver(model_pfr_cy, *opts_setup[0])
    return 0

def run_cy_typed(opts_setup):
    dyn_solver(opts_setup[1][0].cython_model, *opts_setup[1][1])
    return 0

def run_numba(opts_setup):
    dyn_solver(jitted_pfr_model, *opts_setup[2])
    return 0

kernels = [run_np, run_cy_typed, run_numba]

In [ ]:
perfplot.show(
    setup= setups_all,
    kernels= kernels,
    #labels=['py', 'np', 'cy_naive', 'cy_typed', 'numba'],
    labels=['np', 'cy_typed', 'numba'],
    n_range=[10*k for k in range(1, 100+10, 100)], #n_range=[2**k for k in range(1, 100 + 10, 10)],
    xlabel='N'
    )

## Comparing Dasslc wrapper with data copy vs shared memory for the residue vector


In [14]:
def model_pfr_np_no_share(t, y, yp, par):
    res = np.empty(y.size)
    N = par.N
    D, vz, k, Cf, h = par.D, par.vz, par.k, par.Cf, par.h
    dCi = yp
    Ci = y
    aux1 = D / (vz * h)
    C0 = 1.0 / (1.0 + aux1) * (aux1 * Ci[0] + Cf)
    CNp1 = Ci[N - 1]
    aux2 = D / h**2
    aux3 = vz / (2 * h)
    res[0] = aux2 * (Ci[1] - 2.0 * Ci[0] + C0) - \
        aux3 * (Ci[1] - C0) + k * Ci[0] - dCi[0]
    tt1 = aux2 * (Ci[2:] - 2.0 * Ci[1:-1] + Ci[0:-2])
    tt2 = -aux3 * (Ci[2:] - Ci[0:-2]) + k * Ci[1:-1]
    res[1:-1] = tt1 + tt2 - dCi[1:-1]

    res[N - 1] = aux2 * (CNp1 - 2.0 * Ci[N - 1] + Ci[N - 2]) - \
        aux3 * (CNp1 - Ci[N - 2]) + k * Ci[N - 1] - dCi[N - 1]
    return res, 0

def run_np_no_share(opts_setup):
    dasslcy.solve(model_pfr_np_no_share, *opts_setup[0])
    return 0  
    
    
kernels = [run_np, run_np_no_share]

In [ ]:
perfplot.show(
    setup= setups_all,
    kernels= kernels,
    #labels=['py', 'np', 'cy_naive', 'cy_typed', 'numba'],
    labels=['np-shared', 'np-no-shared', 'numba'],
    #n_range=[10*k for k in range(1, 1000+10, 1000)], #n_range=[2**k for k in range(1, 100 + 10, 10)],
    n_range=[10*k for k in np.linspace(1, 10000, 5, dtype='int')],
    xlabel='N'
    )

 20%|██        | 1/5 [00:01<00:06,  1.59s/it]